In [7]:
import geemap
import geopandas
import ee
import os

In [8]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
clipping_shape = 'C:/Users/Jeremy/OneDrive/GIS/Blender/Walkthrough/bounding_box.shp'
clipping = geemap.shp_to_ee(clipping_shape)
clipping_geometry = clipping.geometry()

Map.addLayer(clipping_geometry, {}, 'clipping')

In [10]:
# A function that scales and masks Landsat 8 (C2) surface reflectance images.
# Source: https://gis.stackexchange.com/questions/425159/how-to-make-a-cloud-free-composite-for-landsat-8-collection-2-surface-reflectanc
def prepSrL8(image):
  # Develop masks for unwanted pixels (fill, cloud, cloud shadow).
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  def getFactorImg(factorNames):
    factorList = image.toDictionary().select(factorNames).values()
    return ee.Image.constant(factorList)

  scaleImg = getFactorImg([
    'REFLECTANCE_MULT_BAND_.|TEMPERATURE_MULT_BAND_ST_B10'])
  offsetImg = getFactorImg([
    'REFLECTANCE_ADD_BAND_.|TEMPERATURE_ADD_BAND_ST_B10'])
  scaled = image.select('SR_B.|ST_B10').multiply(scaleImg).add(offsetImg)

  # Replace original bands with scaled bands and apply masks.
  return image.addBands(scaled, None, True) \
    .updateMask(qaMask).updateMask(saturationMask)

In [14]:
# Landsat 8 Collection 2 surface reflectance images of interest.
ls8_comp_per_year = ee.ImageCollection([]); # Create an empty image collection to store composites.

# Define the time range.
startYear = 2020
endYear = 2020
years = list(range(startYear, endYear, 1))

#Loop through years to get a summer cloud free composite of study area
for year in years:

  #Set date range
  startDate = ee.Date.fromYMD(year, 6, 1)
  endDate = ee.Date.fromYMD(year, 8, 31)

  #Generate composite from collection
  composite = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(clipping_geometry) \
    .filterDate(startDate, endDate) \
    .filter(ee.Filter.calendarRange(5, 8, 'month')) \
    .map(prepSrL8) \
    .select('SR_B.|ST_B10') \
    .median() \
    .clip(clipping_geometry)

  #Add year to metadata
  composite = composite.set('year', year)

  #Append year composite to collection
  ls8_comp_per_year = ls8_comp_per_year.merge(composite); # Add the composite to the collection.

In [12]:
#Select images for 3 years
composite2016 = ls8_comp_per_year.filterMetadata('year', 'equals', 2016).first()
composite2018 = ls8_comp_per_year.filterMetadata('year', 'equals', 2018).first()
composite2020 = ls8_comp_per_year.filterMetadata('year', 'equals', 2020).first()

#Plot composite images for three years
Map.addLayer(composite2016, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2016 Composite')
Map.addLayer(composite2018, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2018 Composite')
Map.addLayer(composite2020, {'bands':['SR_B4',  'SR_B3',  'SR_B2'], 'min':0, 'max':0.3}, 'Landsat-8 2020 Composite')

In [13]:
#Create a grid to use for tiling the imagery (due to GEE memory limits...)
grid = geemap.fishnet(clipping_geometry, rows=2, cols=2)
Map.addLayer(grid, {}, 'Grids')

In [18]:
# Specify the parameters for downloading the image
params = {
    'image': composite2020,
    'filename': "C:/Users/Jeremy/OneDrive/GIS/Blender/Walkthrough/Input Landsat Imagery/landsat.tif",
    'scale': 30,
    'crs': 'EPSG:4326',  # Specify the desired projection (e.g., WGS84)
    'region': clipping_geometry  # Specify the region of interest for the download
}
geemap.download_ee_image(**params)

landsat.tif: |          | 0.00/259M (raw) [  0.0%] in 00:00 (eta:     ?)

There is no STAC entry for: None
